# Data Preprocess

## 1. Dataset Preparation

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import random
import datetime as dt
import gc

import seaborn as sns
import matplotlib.pyplot as plt
color = sns.color_palette()

# %matplotlib inline
np.random.seed(1)

In [2]:
# Load the dataset
train = pd.read_csv('../dataset/raw/train_2016_v2.csv' , parse_dates=["transactiondate"]) 
properties = pd.read_csv('../dataset/raw/properties_2016.csv')   
test = pd.read_csv('../dataset/raw/sample_submission.csv') 
test = test.rename(columns = {'ParcelId': 'parcelid'})

# See dataset information
print("Training Data Size:" + str(train.shape))
print("Property Data Size:" + str(properties.shape))
print("Test Data Size:" + str(test.shape))

/home/user/miniconda3/envs/cz4041/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (22,32,34,49,55) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Training Data Size:(90275, 3)
Property Data Size:(2985217, 58)
Test Data Size:(2985217, 7)


In [3]:
# Convert dataset into int32 and float32
for col, dtype in zip(properties.columns, properties.dtypes):
    if dtype == np.float64:        
        properties[col] = properties[col].astype(np.float32)
    if dtype == np.int64:
        properties[col] = properties[col].astype(np.int32)

for col in test.columns:
    if test[col].dtype == int:
        test[col] = test[col].astype(np.int32)
    if test[col].dtype == float:
        test[column] = test[col].astype(np.float32)

## 2. Feature Engineering

In [4]:
properties.head(20)

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.369141,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.570312,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.169922,NaN,NaN,NaN
5,10898347,NaN,NaN,NaN,0.0,0.0,4.0,7.0,NaN,NaN,...,1.0,NaN,176383.0,283315.0,2015.0,106932.0,3661.280029,NaN,NaN,NaN
6,10933547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,397945.0,554573.0,2015.0,156628.0,6773.339844,NaN,NaN,NaN
7,10940747,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,1.0,NaN,101998.0,688486.0,2015.0,586488.0,7857.839844,NaN,NaN,NaN
8,10954547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
9,10976347,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,218440.0,261201.0,2015.0,42761.0,4054.760010,NaN,NaN,NaN


### 2.1. Bathroom Count

In [5]:
drop_columns = set()
bathroom = properties[['bathroomcnt', 'calculatedbathnbr', 'threequarterbathnbr', 'fullbathcnt']]

# See whether each non-null value in calculatedbathnbr is equal to the value in bathroomcnt
bath_count = bathroom[bathroom['calculatedbathnbr'].notnull()][['bathroomcnt', 'calculatedbathnbr']]

print("Column 'calculatedbathnbr'")
print("Number of non-null data: ", bath_count.shape[0])
print("Number of equal data with column 'bathroomcnt': ", 
      sum(bath_count['bathroomcnt'] == bath_count['calculatedbathnbr']))

# Since the value are equal, we can remove the calculatedbathnbr column!
drop_columns.add('calculatedbathnbr')

# See whether 'bathroomcnt' = 0.5 * 'threequarterbathnbr' + 'fullbathcnt'
fractional_bath = bathroom[bathroom['threequarterbathnbr'].notnull()][['bathroomcnt', 'threequarterbathnbr', 'fullbathcnt']]

print("\nCheck wheter 'bathroomcnt' = 0.5 * 'threequarterbathnbr' + 'fullbathcnt'")
print("Number of non-null 'threequarterbathnbr': ", fractional_bath.shape[0])
print("Number of data that satisfied the formula: ", 
     sum(fractional_bath['bathroomcnt'] == (0.5 * fractional_bath['threequarterbathnbr'] + fractional_bath['fullbathcnt'])))

# Since the value are equal, we can remove the 'threequarterbathnbr' column
# and use only 'bathroomcnt' and 'fullbathcnt'
drop_columns.add('threequarterbathnbr')

Column 'calculatedbathnbr'
Number of non-null data:  2856305
Number of equal data with column 'bathroomcnt':  2856305

Check wheter 'bathroomcnt' = 0.5 * 'threequarterbathnbr' + 'fullbathcnt'
Number of non-null 'threequarterbathnbr':  311631
Number of data that satisfied the formula:  311631


### 2.2. Drop Columns

In [6]:
threshold = 0.5 # drop column if number of missing values > threshold
drop_columns = drop_columns.union(
    set(properties.columns[properties.isnull().sum() > threshold * properties.shape[0]]))

### 2.2. Adding More Features

In [7]:
# simple feature engineering

#living area proportions 
properties['living_area_prop'] = properties['calculatedfinishedsquarefeet'] / properties['lotsizesquarefeet']

#tax value ratio
properties['value_ratio'] = properties['taxvaluedollarcnt'] / properties['taxamount']

#tax value proportions
properties['value_prop'] = properties['structuretaxvaluedollarcnt'] / properties['landtaxvaluedollarcnt']

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
corr = properties.corr()
mask = np.triu(corr)
sns.heatmap(corr, ax = ax, annot=True, fmt='.1f', mask=mask, annot_kws={"fontsize":6})

In [8]:
df_train = train.merge(properties, how = 'left', on = 'parcelid') 
df_test = test.merge(properties, how = 'left', on = 'parcelid')

In [ ]:
df_train

In [9]:
###Merging the Datasets ###

# We are merging the properties dataset with training and testing dataset for model building and testing prediction #

df_train = train.merge(properties, how='left', on='parcelid') 
df_test = test.merge(properties, how='left', on='parcelid')


### Remove previos variables to keep some memory
del properties, train
gc.collect();


print('Memory usage reduction...')
df_train[['latitude', 'longitude']] /= 1e6
df_test[['latitude', 'longitude']] /= 1e6

df_train['censustractandblock'] /= 1e12
df_test['censustractandblock'] /= 1e12


### Let's do some pre-exploratory analysis to identify how much missing values do we have in our datasets. 
### Thanks to Nikunj-Carefully dealing with missing values. Ref. https://www.kaggle.com/nikunjm88/carefully-dealing-with-missing-values 

# Let's do some engineering with fireplaceflag variable.

print(df_train.fireplaceflag.isnull().sum())
print(df_train.fireplacecnt.isnull().sum())
# By using fireplacecnt variable we can recover some fields of fireplaceflag

df_train['fireplaceflag']= "No"
df_train.loc[df_train['fireplacecnt']>0,'fireplaceflag']= "Yes"

# Remaining Missing fireplacecnt will be replaced with 0.
index = df_train.fireplacecnt.isnull()
df_train.loc[index,'fireplacecnt'] = 0

#Tax deliquency flag - assume if it is null then doesn't exist
index = df_train.taxdelinquencyflag.isnull()
df_train.loc[index,'taxdelinquencyflag'] = "None"


# Similar step performed for Pool/Spa/hot tub
print(df_train.hashottuborspa.value_counts())
print(df_train.pooltypeid10.value_counts())

#lets remove 'pooltypeid10' as has more missing values
print(df_train.hashottuborspa.value_counts())
print(df_train.pooltypeid10.value_counts())

#Assume if the pooltype id is null then pool/hottub doesnt exist 
index = df_train.pooltypeid2.isnull()
df_train.loc[index,'pooltypeid2'] = 0

index = df_train.pooltypeid7.isnull()
df_train.loc[index,'pooltypeid7'] = 0

index = df_train.poolcnt.isnull()
df_train.loc[index,'poolcnt'] = 0

### Label Encoding For Machine Learning & Filling Missing Values ###

# We are now label encoding our datasets. 
# All of the machine learning algorithms employed in scikit learn assume that 
# the data being fed to them is in numerical form. 
# LabelEncoding ensures that all of our categorical variables are in numerical representation. 
# Also note that we are filling the missing values in our dataset with a zero before label encoding them. 
# This is to ensure that label encoder function does not experience any problems while carrying out its operation 

from sklearn.preprocessing import LabelEncoder  

lbl = LabelEncoder()
for c in df_train.columns:
    df_train[c]=df_train[c].fillna(0)
    if df_train[c].dtype == 'object':
        lbl.fit(list(df_train[c].values))
        df_train[c] = lbl.transform(list(df_train[c].values))

for c in df_test.columns:
    df_test[c]=df_test[c].fillna(0)
    if df_test[c].dtype == 'object':
        lbl.fit(list(df_test[c].values))
        df_test[c] = lbl.transform(list(df_test[c].values))     


### Removing the Outliers

log_errors = df_train['logerror']
df_train = df_train[df_train.logerror < np.percentile(log_errors, 99.5)]
df_train = df_train[df_train.logerror > np.percentile(log_errors, 0.5)]

### Rearranging the DataSets ###

# We will now drop the features that serve no useful purpose. We will also split our data and divide it into the representation to make it clear which features are to be treated as determinants in predicting the outcome for our target feature. Make sure to include the same features in the test set as were included in the training set #
df_train = df_train.drop(drop_columns, axis=1)
x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 
                         'propertycountylandusecode'], axis=1)

df_test = df_test.drop(drop_columns, axis=1)
x_test = df_test.drop(['parcelid', 'propertyzoningdesc',
                       'propertycountylandusecode', '201610', '201611', 
                       '201612', '201710', '201711', '201712'], axis = 1) 

x_train = x_train.values
y_train = df_train['logerror'].values

Memory usage reduction...
90053
80668
True    2365
Name: hashottuborspa, dtype: int64
1.0    1161
Name: pooltypeid10, dtype: int64
True    2365
Name: hashottuborspa, dtype: int64
1.0    1161
Name: pooltypeid10, dtype: int64


In [11]:
x_test.shape

(2985217, 28)

In [37]:
from sklearn.model_selection import train_test_split

X = x_train
y = y_train 

Xtrain, Xvalid, ytrain, yvalid = train_test_split(X, y, test_size=0.2, random_state=42)

###Implement the Xgboost### 

# We can now select the parameters for Xgboost and monitor the progress of results on our validation set. The explanation of the xgboost parameters and what they do can be found on the following link http://xgboost.readthedocs.io/en/latest/parameter.html #

dtrain = xgb.DMatrix(Xtrain, label=ytrain)
dvalid = xgb.DMatrix(Xvalid, label=yvalid)
dtest = xgb.DMatrix(x_test.values)

# Try different parameters! 
xgb_params = {'gpu_id':0, 
              'tree_method':'gpu_hist', 
              'min_child_weight': 3, 
              'eta': 0.01, 
              'colsample_bytree': 0.5, 
              'max_depth': 5,
              'subsample': 0.85, 
              'lambda': 0.9, 
              'nthread': -1, 
              'booster' : 'gbtree', 
              'silent': 1, 
              'gamma' : 0,
              'eval_metric': 'mae', 
              'objective': 'reg:linear'
             }           

watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

model_xgb = xgb.train(xgb_params, dtrain, 1000, watchlist, early_stopping_rounds=100,
                  maximize = False, verbose_eval=10)

###Predicting the results###
# We want to predict oct, nov, dec 2016 (public dataset) and oct, nov, dec 2017 (private dataset)
# Let us now predict the target variable for our test dataset. All we have to do now is just fit the already trained model on the test set that we had made merging the sample file with properties dataset #



[16:25:41] WARNING: ../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[16:25:41] WARNING: ../src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-mae:0.48618	valid-mae:0.48407
[10]	train-mae:0.44021	valid-mae:0.43810
[20]	train-mae:0.39878	valid-mae:0.39667
[30]	train-mae:0.36147	valid-mae:0.35939
[40]	train-mae:0.32789	valid-mae:0.32584
[50]	train-mae:0.29769	valid-mae:0.29569
[60]	train-mae:0.27055	valid-mae:0.26863
[70]	train-mae:0.24619	valid-mae:0.24435
[80]	train-mae:0.22436	valid-mae:0.22260
[90]	train-mae:0.20478	valid-mae:0.20308
[100]	train-mae:0.18727	valid-mae:0.18561
[110]	train-mae:0.17161	valid-mae:0.17000
[120]	train-mae:0.15762	valid-m

In [38]:
Predicted_test_xgb = model_xgb.predict(dtest)

### Submitting the Results ###

# Once again load the file and start submitting the results in each column #
sample_file = pd.read_csv('../dataset/raw/sample_submission.csv') 
for c in sample_file.columns[sample_file.columns != 'ParcelId']:
    sample_file[c] = Predicted_test_xgb

print('Preparing the csv file ...')
sample_file.to_csv('../submission/xgb_predicted_results.csv', index=False, float_format='%.4f')
print("Finished writing the file")

Preparing the csv file ...
Finished writing the file
